In [1]:
!pip install nglview
!pip install pandas
!pip install mdtraj

#import the packages
import nglview as ng
import pandas as pd
import mdtraj as md

In [2]:
# %cd /home/emmmb/jupyter-env/docking/data ## For PC

%cd /Users/emmamorris/Desktop/HADDOCK/MgCl2/Data

/Users/emmamorris/Desktop/HADDOCK/MgCl2/Data


In [3]:
%ls

#cluster5_1_processed.gro.1#  Input/
#final_ions.gro.1#            aptamer_box.gro
#ions_nacl.tpr.1#             aptamer_processed.gro
#ions_nacl.tpr.2#             cluster5_1_processed.gro
#posre.itp.1#                 final_ions.gro
#posre.itp.2#                 ions.mdp
#topol.top.1#                 ions.tpr
#topol.top.2#                 ions_nacl.tpr
#topol.top.3#                 mdout.mdp
#topol.top.4#                 minim.mdp
#topol.top.5#                 nvt.mdp
#topol.top.6#                 posre.itp
CD63_aptamer.pdb              solv_mg.gro
CD63_protein.pdb              solvated.gro
CD63_tmp.pdb                  topol.top


In [4]:
view = ng.show_structure_file("Input/CD63_Xiao_14072025.pdb")
view

ValueError: you must provide file extension if using file-like object or text content

In [ ]:
!grep -v HETATM Input/CD63_Xiao_14072025.pdb > CD63_tmp.pdb
!grep -v CONECT CD63_tmp.pdb > CD63_aptamer.pdb

In [ ]:
view = ng.show_structure_file("CD63_aptamer.pdb")
view

In [ ]:
!gmx pdb2gmx -f CD63_protein.pdb -o aptamer_processed.gro -p topol.top -water tip3p -ff amber99sb-ildn -ignh

In [ ]:
!ls

In [ ]:
!gmx editconf -f aptamer_processed.gro -o aptamer_box.gro -c -d 1.0 -bt dodecahedron

In [ ]:
!gmx solvate -cp aptamer_box.gro -cs spc216.gro -o solvated.gro -p topol.top

In [ ]:
!touch ions.mdp

In [ ]:
!gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr

In [ ]:
!printf "SOL\n" | gmx genion -s ions.tpr -o solv_mg.gro -p topol.top -pname MG -nname CL -conc 0.005 -neutral


In [ ]:
!gmx grompp -f ions.mdp -c solv_mg.gro -p topol.top -o ions_nacl.tpr

!printf "SOL\n" | gmx genion -s ions_nacl.tpr -o final_ions.gro -p topol.top -pname NA -nname CL -conc 0.14


In [ ]:
view = ng.show_structure_file("solvated.gro")
view.add_representation(repr_type='ball+stick', selection='SOL')
view.camera='orthographic'
view

In [ ]:
# Load gromacs file
view = ng.show_structure_file("final_ions.gro")

# Represent ions clearly
view.add_representation(repr_type='spacefill', selection='NA', color='blue')
view.add_representation(repr_type='spacefill', selection='CL', color='green')
view.add_representation(repr_type='spacefill', selection='MG', color='red')

# Represent water as ball+stick
view.add_representation(repr_type='ball+stick', selection='SOL')

# Show aptamer as cartoon or licorice if desired
view.add_representation(repr_type='cartoon', selection='nucleic')

# Set camera
view.camera='orthographic'

view

## Energy minimisation

In [ ]:
with open("minim.mdp", "w") as f:
    f.write("""
; minim.mdp - Parameters for energy minimization

integrator    = steep       ; Algorithm (steepest descent minimization)
emtol         = 1000.0      ; Stop minimization when max force < 1000.0 kJ/mol/nm
emstep        = 0.01        ; Initial step-size
nsteps        = 50000       ; Maximum number of minimization steps

cutoff-scheme = Verlet
nstlist       = 10
ns_type       = grid
rlist         = 1.2
rcoulomb      = 1.2
rvdw          = 1.2

coulombtype   = PME         ; Particle Mesh Ewald for long-range electrostatics
pbc           = xyz         ; Periodic boundary conditions in all directions
constraints   = none        ; No constraints during minimization
""")

In [ ]:
!gmx grompp -f minim.mdp -c final_ions.gro -p topol.top -o em.tpr
!gmx mdrun -deffnm em -v

In [ ]:
!printf "potential\n0\n" | gmx energy -f em.edr -o potential.xvg -xvg none

In [ ]:
df = pd.read_csv('potential.xvg', sep='\\s+', header=None, names=['step', 'energy])
df.plot('step')                                                                  

 ## NVT (temperature) equilibration

In [ ]:
with open("nvt.mdp", "w") as f:
    f.write("""
; NVT equilibration for aptamer
title                    = NVT equilibration
define                   = -DPOSRES        ; position restraints (optional)
integrator               = md              ; leap-frog integrator
nsteps                   = 50000           ; 100 ps if timestep is 2 fs
dt                       = 0.002           ; 2 fs

; Output control
nstxout                  = 1000
nstvout                  = 1000
nstenergy                = 500
nstlog                   = 500
nstxout-compressed       = 1000

; Bond parameters
continuation             = no              ; starting fresh from minimization
constraint_algorithm     = lincs
constraints              = h-bonds
lincs_iter               = 1
lincs_order              = 4

; Neighbor searching
cutoff-scheme            = Verlet
ns_type                  = grid
nstlist                  = 10
rcoulomb                 = 1.0
rvdw                     = 1.0

; Electrostatics
coulombtype              = PME
pme_order                = 4
fourierspacing           = 0.16

; Temperature coupling is on
tcoupl                   = V-rescale                     ; modified Berendsen thermostat
tc-grps                  = System
tau_t                    = 0.1
ref_t                    = 300

; Pressure coupling is off during NVT
pcoupl                   = no

; Periodic boundary conditions
pbc                      = xyz

; Dispersion correction
DispCorr                 = EnerPres

; Velocity generation
gen_vel                  = yes
gen_temp                 = 300
gen_seed                 = -1
""")

In [ ]:
!gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr
!gmx mdrun -deffnm nvt

Analyse NVT Equil

In [ ]:
!echo "Temperature" | gmx energy -f nvt.edr -o temperature.xvg -xvg none -b 20

In [ ]:
df = pd.read_csv('temperature.xvg', sep='\\s+', header=None, names=['time','temperature'])
df.plot('time')

## Pressure - NPT Equil (100 ps)

In [ ]:
with open("npt.mdp", "w") as f:
    f.write("""
; title                    = NPT equilibration
define                   = -DPOSRES        ; Position restraints for solute
integrator               = md
dt                       = 0.002           ; 2 fs time step
nsteps                   = 50000           ; 100 ps
refcoord_scaling = com

; Output control
nstxout                  = 1000
nstvout                  = 1000
nstenergy                = 1000
nstlog                   = 1000

; Bond parameters
continuation             = yes             ; Continue from NVT
constraint_algorithm     = lincs
constraints              = h-bonds
lincs_iter               = 1
lincs_order              = 4

; Neighborsearching
cutoff-scheme            = Verlet
ns_type                  = grid
nstlist                  = 10
rcoulomb                 = 1.0
rvdw                     = 1.0

; Electrostatics
coulombtype              = PME
pme_order                = 4
fourierspacing           = 0.16

; Temperature coupling
tcoupl                   = V-rescale
tc-grps                  = System
tau_t                    = 0.1
ref_t                    = 300

; Pressure coupling
pcoupl                   = Parrinello-Rahman
pcoupltype               = isotropic
tau_p                    = 2.0
ref_p                    = 1.0
compressibility          = 4.5e-5

; Velocity generation
gen_vel                  = no              ; Keep velocities from NVT
""")

In [ ]:
!gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr
!gmx mdrun -deffnm npt

Visualise results

In [ ]:
!echo "Pressure" | gmx energy -f npt.edr -o pressure.xvg -xvg none

In [ ]:
df = pd.read_csv('pressure.xvg', sep='\\s+', header=None, names=['time','pressure'])
df.plot('time')

# Production run (50 ns) THE BIG ONE

In [ ]:
with open("md.mdp", "w") as f:
    f.write("""
; title                    = Production MD
integrator               = md
dt                       = 0.002              ; 2 fs
nsteps                   = 25000000         ; 50 ns = 50,000 ps / 0.002 ps = 25,000,000 steps

; Output control
nstxout                  = 10000              ; write coordinates every 20 ps
nstvout                  = 0                  ; don't write velocities
nstenergy                = 1000               ; write energies every 2 ps
nstlog                   = 1000               ; write to log every 2 ps
nstxout-compressed       = 1000               ; compressed trajectory output

; Bond parameters
continuation             = yes
constraint_algorithm     = lincs
constraints              = h-bonds
lincs_iter               = 1
lincs_order              = 4

; Neighborsearching
cutoff-scheme            = Verlet
nstlist                  = 10
rcoulomb                 = 1.0
rvdw                     = 1.0

; Electrostatics
coulombtype              = PME
pme_order                = 4
fourierspacing           = 0.16

; Temperature coupling
tcoupl                   = V-rescale
tc-grps                  = System
tau_t                    = 0.1
ref_t                    = 300

; Pressure coupling
pcoupl                   = Parrinello-Rahman
pcoupltype               = isotropic
tau_p                    = 2.0
ref_p                    = 1.0
compressibility          = 4.5e-5

; Velocity generation
gen_vel                  = no                 ; velocities come from equilibration
""")

In [ ]:
!gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr
!gmx mdrun -v -deffnm md

# Analysing final folded aptamer

In [ ]:
#echo "1 1"
!printf "1\n1\n" | gmx trjconv -s md.tpr -f md.xtc -o md_center.xtc -center -pbc mol

In [ ]:
### Tutorial ###
# traj = md.load("md_center.xtc", top="md.gro") ### THIS WORKS - missing aptamer
traj = md.load("md_center.xtc", top="aptamer_box.gro")
view = ng.show_mdtraj(traj)
view

In [ ]:
traj = md.load("md_center.xtc", top="aptamer_box.gro")
view = ng.show_mdtraj(traj)
view.clear_representations()
view.add_cartoon(selection="protein")     # Protein cartoon
view.add_licorice(selection="nucleic")    # Aptamer backbone in full
view.add_ball_and_stick(selection="not protein and not nucleic")  # Everything else
view

## RMSD Overtime

In [ ]:
import mdtraj as md
import matplotlib.pyplot as plt

# Load the trajectory
traj = md.load("md_center.xtc", top="aptamer_box.gro")

# Manually filter for RNA backbone atoms (P, C4', C3', O3')
backbone_atom_names = ["P", "C4'", "C3'", "O3'"]

# Get indices of backbone atoms that belong to standard nucleotide residues
rna_bb = [atom.index for atom in traj.topology.atoms
          if atom.name in backbone_atom_names and atom.residue.is_nucleic]

# Align to the first frame using RNA backbone atoms
traj.superpose(traj[0], atom_indices=rna_bb)

# Calculate RMSD
rmsd = md.rmsd(traj, traj, 0, atom_indices=rna_bb)

# Plot
plt.plot(traj.time / 1000, rmsd)
plt.xlabel("Time (ns)")
plt.ylabel("RMSD (nm)")
plt.title("Aptamer Backbone RMSD")
plt.grid(True)
plt.show()

## H-bonding

In [ ]:
# Load trajectory (example, adjust as needed)
# traj = md.load("trajectory.xtc", top="topology.pdb")

# Select only nucleic acid atoms
nucleic_atoms = traj.topology.select("nucleic")

# Compute all hydrogen bonds
hbonds = md.baker_hubbard(traj, periodic=True)

# Filter for only nucleic–nucleic hydrogen bonds
nucleic_hbonds = [
    hbond for hbond in hbonds
    if hbond[0] in nucleic_atoms and hbond[2] in nucleic_atoms
]

print(f"Nucleic–nucleic hydrogen bonds detected: {len(nucleic_hbonds)}")

# Print first few
for hbond in nucleic_hbonds[:10]:
    donor, hydrogen, acceptor = hbond
    print(f"Donor: {traj.topology.atom(donor)}, Acceptor: {traj.topology.atom(acceptor)}")


Lost and gained H-bonds

In [ ]:
import mdtraj as md

# Load trajectory
#traj = md.load("trajectory.xtc", top="aptamer_box.gro")

# Select only nucleic atoms
nucleic_atoms = traj.topology.select("nucleic")

# Define frame indices
frame_start = traj[0]
frame_end = traj[-1]

# Compute H-bonds for first and last frame
hbonds_start = md.baker_hubbard(frame_start, periodic=True)
hbonds_end = md.baker_hubbard(frame_end, periodic=True)

# Filter: only nucleic-nucleic
nuc_hbonds_start = set(
    (d, a) for d, h, a in hbonds_start
    if d in nucleic_atoms and a in nucleic_atoms
)
nuc_hbonds_end = set(
    (d, a) for d, h, a in hbonds_end
    if d in nucleic_atoms and a in nucleic_atoms
)

# Compare sets
common = nuc_hbonds_start & nuc_hbonds_end
lost = nuc_hbonds_start - nuc_hbonds_end
gained = nuc_hbonds_end - nuc_hbonds_start

print(f"Initial nucleic–nucleic H-bonds: {len(nuc_hbonds_start)}")
print(f"Final nucleic–nucleic H-bonds: {len(nuc_hbonds_end)}")
print(f"Shared H-bonds: {len(common)}")
print(f"Lost H-bonds: {len(lost)}")
print(f"Gained H-bonds: {len(gained)}\n")

# Optional: print actual donor–acceptor atom info
def describe_hbonds(hbond_set, label):
    print(f"{label}:")
    for donor, acceptor in hbond_set:
        print(f"  Donor: {traj.topology.atom(donor)}, Acceptor: {traj.topology.atom(acceptor)}")
    print()

describe_hbonds(lost, "Lost H-bonds")
describe_hbonds(gained, "Gained H-bonds")

H bonds pre run

In [ ]:
# Load only the first frame of the trajectory
frame0 = md.load("md_center.xtc", top="aptamer_box.gro", frame=0)

# Select nucleic atoms
nucleic_atoms = frame0.topology.select("nucleic")

# Compute H-bonds for frame 0 only
hbonds_start = md.baker_hubbard(frame0, periodic=True)

# Filter nucleic–nucleic H-bonds
nucleic_hbonds_start = [
    hbond for hbond in hbonds_start
    if hbond[0] in nucleic_atoms and hbond[2] in nucleic_atoms
]

print(f"Nucleic–nucleic H-bonds at start: {len(nucleic_hbonds_start)}")

# Print some
for donor, hydrogen, acceptor in nucleic_hbonds_start[:10]:
    print(f"Donor: {frame0.topology.atom(donor)}, Acceptor: {frame0.topology.atom(acceptor)}")

H bonds post run

In [ ]:
import mdtraj as md

# Load full trajectory
traj = md.load("md_center.xtc", top="aptamer_box.gro")

# Extract the last frame
frame_end = traj[-1]

# Select only nucleic acid atoms in the final frame
nucleic_atoms_end = frame_end.topology.select("nucleic")

# Compute H-bonds in the final frame only
hbonds_end = md.baker_hubbard(frame_end, periodic=True)

# Filter nucleic–nucleic H-bonds
nucleic_hbonds_end = [
    hbond for hbond in hbonds_end
    if hbond[0] in nucleic_atoms_end and hbond[2] in nucleic_atoms_end
]

print(f"Nucleic–nucleic H-bonds at end: {len(nucleic_hbonds_end)}")

# Print first few
for donor, hydrogen, acceptor in nucleic_hbonds_end[:10]:
    print(f"Donor: {frame_end.topology.atom(donor)}, Acceptor: {frame_end.topology.atom(acceptor)}")
